In [1]:
import tensorflow_datasets as tfds
import tensorflow as tf

In [2]:
tfds.disable_progress_bar()

raw_train, raw_test = tfds.load('cats_vs_dogs',split=['train[:80%]','train[20%:]'], as_supervised=True)

In [3]:
tfds.features.FeaturesDict({
    'image': tfds.features.Image(shape=(None, None, 3), dtype=tf.uint8),
    'image/filename': tfds.features.Text(),  # Text type for filename
    'label': tfds.features.ClassLabel(num_classes=2),})

FeaturesDict({
    'image': Image(shape=(None, None, 3), dtype=uint8),
    'image/filename': Text(shape=(), dtype=string),
    'label': ClassLabel(shape=(), dtype=int64, num_classes=2),
})

In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.image import ResizeMethod

In [5]:
def preprocess(image,label):
    out_image = tf.image.resize(image,[244,244],method=ResizeMethod.BICUBIC)
    out_image = tf.keras.applications.mobilenet_v2.preprocess_input(out_image)
    return out_image, label
batch_size = 32
train_batch = raw_train.map(preprocess).batch(batch_size)
test_batch = raw_test.map(preprocess).batch(batch_size)

In [6]:
mobilenet_base = tf.keras.applications.MobileNetV2(
    input_shape=(244, 244, 3),  # Input size should be 224x224
    weights="imagenet",
    include_top=False)
mobilenet_base.trainable = False

mobilenet_model = tf.keras.Sequential([
    mobilenet_base,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1)
])


In [8]:
mobilenet_model.compile(optimizer='adam',
                        loss='binary_crossentropy',
                        metrics=['accuracy'])

In [10]:
mobilenet_model.fit(train_batch,epochs=5)

Epoch 1/5
582/582 [==============================] - 368s 628ms/step - loss: 0.2645 - accuracy: 0.9633
Epoch 2/5
582/582 [==============================] - 368s 632ms/step - loss: 0.1443 - accuracy: 0.9827
Epoch 3/5
582/582 [==============================] - 368s 633ms/step - loss: 0.1830 - accuracy: 0.9811
Epoch 4/5
582/582 [==============================] - 367s 630ms/step - loss: 0.1325 - accuracy: 0.9853
Epoch 5/5
582/582 [==============================] - 366s 628ms/step - loss: 0.1035 - accuracy: 0.9896


In [12]:
mobilenet_model.evaluate(test_batch,verbose=2)

582/582 - 371s - loss: 0.0939 - accuracy: 0.9911 - 371s/epoch - 637ms/step


[0.09388027340173721, 0.9910800457000732]